# Keras Implementation - Human Data Analytics Project 

Importing all modules that we need for our elaboration

In [21]:
import numpy as np
import scipy.io

from keras import regularizers

import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=FutureWarning)
    
    from keras.layers import Conv1D, BatchNormalization, Dropout, LeakyReLU, Flatten, Activation, Dense, MaxPooling1D
    from keras.models import Model, Sequential
    from keras.optimizers import Adam
    from sklearn.preprocessing import OneHotEncoder, StandardScaler
    from sklearn.metrics import f1_score
    import keras.backend as K
    
K.set_image_data_format('channels_last')

%matplotlib inline

# Preprocessing Section

In [2]:
x1 = scipy.io.loadmat("data_temp/S1-ADL1", mdict={'filled_features':'features', 'labels':'labels'})
x2 = scipy.io.loadmat("data_temp/S1-ADL2", mdict={'filled_features':'features', 'labels':'labels'})
x3 = scipy.io.loadmat("data_temp/S1-ADL3", mdict={'filled_features':'features', 'labels':'labels'})
x4 = scipy.io.loadmat("data_temp/S1-ADL4", mdict={'filled_features':'features', 'labels':'labels'})
x5 = scipy.io.loadmat("data_temp/S1-ADL5", mdict={'filled_features':'features', 'labels':'labels'})

We split the data into train and test set.

In [3]:
X_train = np.concatenate((x1['filled_features'],x2['filled_features'],x3['filled_features'],x5['filled_features']),axis=0)
#X_test = np.concatenate((x4['filled_features'],x5['filled_features']),axis=0)
X_test = x4['filled_features']

Y_train = np.concatenate((x1['labels'][:,0],x2['labels'][:,0],x3['labels'][:,0],x5['labels'][:,0]),axis=0) # here we take just the first column because we want to fulfill the first task
Y_test = x4['labels'][:,0]
#Y_test = np.concatenate((x4['labels'][:,0],x5['labels'][:,0]),axis=0)

# Initial decision to overcome the problem of entire missing columns
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)


X_train shape:  (132281, 113)
X_test shape:  (30228, 113)


The dataset needs to be normalized in order to improve performances.

In [4]:
# fatures normalization
scaler = StandardScaler().fit(X_train)
X_train =scaler.transform(X_train)
X_test = scaler.transform(X_test)

We decided to use a onehot encoder for the labels

In [5]:
onehot_encoder = OneHotEncoder(sparse=False)
YOH_train = onehot_encoder.fit_transform(Y_train.reshape(-1, 1))
YOH_test = onehot_encoder.fit_transform(Y_test.reshape(-1, 1))

print("YOH_train shape: ", YOH_train.shape)
print("YOH_test shape: ", YOH_test.shape)

YOH_train shape:  (132281, 5)
YOH_test shape:  (30228, 5)


In [6]:
window_size = 10
stride = int(window_size / 2)
num_features = X_train.shape[1]
samples, classes = YOH_train.shape
print("Number of samples: ", samples, "\nNumber of classes: ", classes)

windows = int(samples // stride) - 1
print("Number of windows: ", windows)

Number of samples:  132281 
Number of classes:  5
Number of windows:  26455


In [7]:
num_features_test = X_test.shape[1]
samples_test, classes_test = YOH_test.shape
print("Number of samples: ", samples_test, "\nNumber of classes: ", classes_test)

windows_test = int(samples_test // stride) - 1
print("Number of windows: ", windows_test)

Number of samples:  30228 
Number of classes:  5
Number of windows:  6044


In [8]:
cube = np.zeros([windows, window_size, num_features])
cube.shape

(26455, 10, 113)

In [9]:
cube_test = np.zeros([windows_test, window_size, num_features])
cube_test.shape

(6044, 10, 113)

In [10]:
lab_cum = np.zeros([windows])
lab_cum.shape

(26455,)

In [11]:
lab_cum_test = np.zeros([windows_test])
lab_cum_test.shape

(6044,)

In [12]:
for w in range(windows):
    index = int(w * stride)
    cube[w,:,:] = X_train[index:index+window_size, :].reshape((window_size,num_features))
    l = YOH_train[index:index+window_size,:] # shape 50 x 5
    #print(np.sum(l, axis=0),np.argmax(np.sum(l, axis=0)))
    lab_cum[w] = np.argmax(np.sum(l, axis=0))

In [13]:
for w in range(windows_test):
    index = int(w * stride)
    cube_test[w,:,:] = X_test[index:index+window_size, :].reshape((window_size,num_features))
    l = YOH_test[index:index+window_size,:] # shape 50 x 5
    #print(np.sum(l, axis=0),np.argmax(np.sum(l, axis=0)))
    lab_cum_test[w] = np.argmax(np.sum(l, axis=0))

In [14]:
#print('cumulative labels: ', lab_cum.shape, type(lab_cum), "\n", lab_cum)
YOH_train_final = onehot_encoder.fit_transform(lab_cum.reshape(-1, 1))
print(YOH_train_final)

[[0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]


In [15]:
YOH_test_final = onehot_encoder.fit_transform(lab_cum_test.reshape(-1, 1))
print(YOH_test_final)

[[0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]


In [16]:
def TestModel(input_shape):
    """ 
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns: 
    model -- a Model() instance in Keras
    """
    
    model = Sequential()
    model.add(Conv1D(filters = 18,
                    kernel_size=5,
                    strides=1,
                    padding='same',
                    input_shape = input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2,
                          strides=2,
                          padding='same'))
    
    model.add(Conv1D(filters = 36,
                    kernel_size=7,
                    strides=1,
                    padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2,
                          strides=2,
                          padding='same'))
    
    model.add(Dropout(0.2))
    
    model.add(Conv1D(filters = 72,
                    kernel_size=7,
                    strides=1,
                    padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=2,
                          strides=2,
                          padding='same'))
    
    #model.add(Conv1D(filters = 144,
    #                kernel_size=7,
    #                strides=1,
    #                padding='same'))
    #model.add(BatchNormalization())
    #model.add(Activation('relu'))
    #model.add(MaxPooling1D(pool_size=2,
    #                      strides=2,
    #                      padding='same'))
    
    model.add(Flatten())
    
    model.add(Dense(64, kernel_regularizer=regularizers.l2(0.01)))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.4))

    model.add(Dense(5))
    model.add(Activation('softmax'))
    
    #model.summary()
    
    return model

In [17]:
model_test = TestModel((10,113))

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [18]:
opt = Adam(lr=0.01)
model_test.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [19]:
model_test.fit(x = cube, y = YOH_train_final, epochs = 50, batch_size = 128, validation_data=(cube_test,YOH_test_final))

Train on 26455 samples, validate on 6044 samples
Epoch 1/50
26455/26455 [==============================] - 6s 214us/step - loss: 0.5426 - acc: 0.8478 - val_loss: 0.4261 - val_acc: 0.8493
Epoch 2/50
26455/26455 [==============================] - 5s 190us/step - loss: 0.3424 - acc: 0.8961 - val_loss: 0.3521 - val_acc: 0.8709
Epoch 3/50
26455/26455 [==============================] - 5s 195us/step - loss: 0.3023 - acc: 0.9086 - val_loss: 0.3478 - val_acc: 0.8858
Epoch 4/50
26455/26455 [==============================] - 5s 189us/step - loss: 0.2696 - acc: 0.9190 - val_loss: 0.2848 - val_acc: 0.9156
Epoch 5/50
26455/26455 [==============================] - 5s 181us/step - loss: 0.2632 - acc: 0.9237 - val_loss: 0.2674 - val_acc: 0.9108
Epoch 6/50
26455/26455 [==============================] - 5s 180us/step - loss: 0.2306 - acc: 0.9296 - val_loss: 0.3146 - val_acc: 0.9097
Epoch 7/50
26455/26455 [==============================] - 5s 180us/step - loss: 0.2330 - acc: 0.9315 - val_loss: 0.2996 - v

In [22]:
y_pred = model_test.predict(cube_test)
print(y_pred)

y_pred_hard = np.argmax(y_pred, axis=1)
print(y_pred_hard)

YOH_pred = onehot_encoder.fit_transform(y_pred_hard.reshape(-1, 1))
print(YOH_pred)

print(f1_score(YOH_test_final, YOH_pred, average='weighted'))

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(classes):
    fpr[i], tpr[i], _ = roc_curve(YOH_test_final[:, i], YOH_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

print(roc_auc)

[[1.7493697e-02 7.3277545e-01 2.4958846e-01 1.1561138e-04 2.6799451e-05]
 [1.3798050e-02 2.9496205e-01 6.9098580e-01 1.8834096e-04 6.5763816e-05]
 [1.2922069e-02 2.6211131e-01 7.2477567e-01 1.4491745e-04 4.6036865e-05]
 ...
 [2.2599554e-02 7.1870947e-01 2.5842422e-01 1.9807353e-04 6.8594250e-05]
 [1.8087987e-02 6.6227216e-01 3.1945768e-01 1.4249918e-04 3.9683062e-05]
 [4.4127894e-03 9.8019397e-01 1.5387553e-02 5.6272202e-06 8.8833822e-08]]
[1 2 2 ... 1 1 1]
[[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 ...
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]
0.9070580936852101


NameError: name 'roc_curve' is not defined